# Marketing Analysis with NLP

This notebook analyzes customer reviews using NLP techniques: preprocessing, sentiment analysis, topic modeling (LDA), and keyword extraction (TF-IDF). It also saves simple visualizations and prints business recommendations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Optional: you can comment seaborn if you prefer pure matplotlib visuals
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from textblob import TextBlob

# Ensure NLTK resources are downloaded
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# 1) Load dataset
df = pd.read_csv('reviews.csv')

# 2) Preprocess text
def preprocess_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text.lower())
    sw = set(stopwords.words('english'))
    tokens = [t for t in tokens if t.isalpha() and t not in sw]
    return " ".join(tokens)

df['cleaned'] = df['review'].apply(preprocess_text)

# 3) Sentiment analysis
def get_sentiment(text: str) -> float:
    try:
        return TextBlob(text).sentiment.polarity
    except Exception:
        return 0.0

df['sentiment'] = df['cleaned'].apply(get_sentiment)
df['sentiment_label'] = df['sentiment'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))

# 4) Topic modeling with LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
dt_matrix = vectorizer.fit_transform(df['cleaned'])

n_topics = 2 if dt_matrix.shape[0] < 10 else 3
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(dt_matrix)

def extract_topics(model, feature_names, topn=8):
    topics = []
    for topic in model.components_:
        terms = [feature_names[i] for i in topic.argsort()[:-topn-1:-1]]
        topics.append(terms)
    return topics

topics = extract_topics(lda, vectorizer.get_feature_names_out(), topn=8)

# 5) Keyword extraction using TF-IDF
tfidf_vec = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vec.fit_transform(df['cleaned'])
keywords = np.array(tfidf_vec.get_feature_names_out())
mean_tfidf = tfidf.mean(axis=0).A1
top_n = 12
top_keywords = keywords[mean_tfidf.argsort()[::-1][:top_n]]

# 6) Visualizations (saved as files)
# Sentiment bar chart
sent_counts = df['sentiment_label'].value_counts().reindex(['negative','neutral','positive']).fillna(0)
plt.figure()
sent_counts.plot(kind='bar')
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('sentiment_distribution.png')

# Word Cloud
all_words = " ".join(df['cleaned'].dropna().astype(str).tolist())
if all_words.strip():
    wc = WordCloud(width=800, height=400, background_color='white').generate(all_words)
    plt.figure()
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud of Reviews')
    plt.tight_layout()
    plt.savefig('wordcloud.png')

# 7) Print insights
print("Sample Reviews with Sentiment:")
print(df[['review', 'sentiment_label']])

print("\nTop Topics Identified:")
for i, t in enumerate(topics, 1):
    print(f"Topic {i}: {', '.join(t)}")

print("\nTop Keywords:")
print(", ".join(top_keywords))

# 8) Business recommendations
recommendations = [
    "- Improve customer service responsiveness to reduce negative mentions.",
    "- Highlight product quality in marketing campaigns.",
    "- Address packaging concerns raised in reviews.",
    "- Leverage positive testimonials in ads and landing pages."
]
print("\nBusiness Recommendations:")
print("\n".join(recommendations))
